In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=5cf0e18fdcd8052243b5268cd13d48e2cdeb68ebe67262e1b871a985920a73ff
  Stored in directory: /Users/taiki/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [28]:
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
def get_info_from_archives_page(url, df):
    url = url
    response = request.urlopen(url)
    bs = BeautifulSoup(response)
    a_all = bs.find_all("a", class_="title")
    for a in a_all:
        a_href = a.attrs["href"]
        df = get_research_paper(a_href, df)
    
    response.close()
    return df

In [42]:
def get_abst_kw(url):
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    if soup is None:
        ab = "N/A"
        kw = "N/A"
    else:
        ab = soup.find("p").text
        kw = soup.find("section", class_="keywords")
        if kw is None:
            kw = "N/A"
        else:
            kw = kw.find("span", class_="value").text.replace("\n", "").replace("\t", "")
            print(kw)
            break
    ab_kw_dict = {"abstract": ab, "keywords": kw}
    return ab_kw_dict

SyntaxError: 'break' outside loop (<ipython-input-42-f53923c75daa>, line 15)

In [43]:
def get_research_paper(url, df):
    url = url
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    volume = soup.find("h1").text.replace("\n", "").replace("\t", "")
    obj_articles = soup.find_all("div", class_="obj_article_summary")
    for obj in obj_articles:
        title = obj.find("h3", class_="title").find("a").text.replace("\n", "").replace("\t", "")
        authors = obj.find("div", class_="authors").text.replace("\n", "").replace("\t", "")
        link = obj.find("a", class_="obj_galley_link pdf")
        if link is None:
            link = "N/A"
        else:
            link = link.attrs["href"]
        article_page = obj.find("a").attrs["href"]
        ab_kw_dict = get_abst_kw(article_page)
        info_dict = {
            "volume": volume,
            "title": title,
            "authors": authors,
            "link": link
        }
        info_dict.update(ab_kw_dict)
        
        df = df.append(info_dict, ignore_index=True, sort=False)
    
    response.close()
    return df


In [44]:
df_paper = pd.DataFrame(columns = ["volume", "title", "authors", "link"])
url = 'https://bop.unibe.ch/JEMR/issue/archive'
while True:
    response = request.urlopen(url)
    bs = BeautifulSoup(response)
    df_paper = get_info_from_archives_page(url, df_paper)
    url = bs.find("a", class_="next")
    if  url is None:
        break
    else:
        url = bs.find("a", class_="next").attrs["href"]
response.close()
df_paper.to_excel("JEMR_title_list.xlsx")